## Creating a shapefile of tiles from a list of geotiff files

In [8]:
import rasterio
import os
from shapely.geometry import box
import geopandas as gpd

# Output must end in `.shp`
def create_tile_shapefile(directory_with_tiled_data, output_shapefile_path):
    # Get a list of all the file names
    files = os.listdir(directory_with_tiled_data)
    
    # Create variables for storing the bounding boxes and ids
    id = 0
    boxes = []
    ids = []
    filenames = []
    for file in files:
        if file.endswith('.tif'):
            filepath = directory_with_tiled_data + file
            
            # Read the file, gather the coordinate reference system and bounds and add them to a running list
            with rasterio.open(filepath) as src:
                crs = src.crs
                bounds = src.bounds
                boxes.append(box(bounds.left, bounds.bottom, bounds.right, bounds.top))
                ids.append(id)
                filenames.append(file)
                id += 1
    
    # Assume the CRS for all of the files is the same and gather it
    crs_wkt = crs.to_wkt()

    # Create a geopandas geodataframe with one entry for each image
    data = {
        'id': ids,
        'filename': filenames,
        'geometry': boxes
    }
    df = gpd.GeoDataFrame(data, crs=crs_wkt)
    
    # Write that to shapefile
    path = 'test.geojson'
    df.to_file(output_shapefile_path)

In [9]:
directory_with_tiled_data = 'tiles/'
output_shapefile_path = 'tile_shapefile/mytiles.shp'
create_tile_shapefile(directory_with_tiled_data, output_shapefile_path)
    